In [80]:
import serial
from struct import *
import time

class MotorModuleController():
    def __init__(self, port):
        try:
            self.ser = serial.Serial(port, timeout= 0.05)
            self.ser.baudrate = 115200
            self.rx_data = [0, 0, 0, 0, 0, 0]
            self.rx_val = [0, 0, 0, 0]
            self.tx_data = [0, 0, 0, 0, 0, 0, 0, 0]
            print('connceted to motor module controller')
        except:
            print('failed to connect to motor module controller')
            pass

        self.P_MAX = 12.5
        self.P_MIN = -12.5
        self.V_MAX = 65.0
        self.V_MIN = -65.0
        self.KP_MAX = 500
        self.KP_MIN = 0
        self.KD_MAX = 5
        self.KD_MIN = 0
        self.I_MAX = 40

    def send_data(self):
        pass


    def pack_cmd(self, id, p_des, v_des, kp, kd, i_ff):
        p_des = self.float2uint(p_des, self.P_MIN, self.P_MAX, 16)
        v_des = self.float2uint(v_des, self.V_MIN, self.V_MAX, 12)
        kp = self.float2uint(kp, self.KP_MIN, self.KP_MAX, 12)
        kd = self.float2uint(kd, self.KD_MIN, self.KD_MAX, 12)
        i_ff = self.float2uint(i_ff, -self.I_MAX, self.I_MAX, 12)

        d1 = p_des >> 8
        d2 = p_des&0xFF
        d3 = v_des>>4
        d4 = ((v_des&0xF)<<4) | (kp>>8)
        d5 = kp&0xFF
        d6 = kd>>4
        d7 = ((kd&0xF)<<4) | i_ff>>8
        d8 = i_ff&0xFF
        
        id = int(id)
        b = bytes(bytearray([id, d1, d2, d3, d4, d5, d6, d7, d8]))
        return b 

    def send_cmd(self, cmd):
        """
        sends data 
        """
        self.ser.write(cmd)
        print('cmd is sent: ', cmd)

    def get_rx(self):
        b_rx = self.ser.read(13)
        self.rx_val[0] = b_rx[0]                   # id
        self.rx_val[1] = unpack("f", b_rx[1:5])    # position
        self.rx_val[2] = unpack("f", b_rx[5:9])    # velocity
        self.rx_val[3] = unpack("f", b_rx[9:13])   # current
        print('recieved: ', self.rx_val)
        return self.rx_val

    def float2uint(self, x, x_min, x_max, bits):
        span = x_max - x_min
        offset = x_min
        return int((x-offset)*(float((1<<bits)-1))/span)

    def uint2float(self, x, x_min, x_max, bits):
        span = x_max - x_min
        offset = x_min
        return int(x*span/((1<<bits)-1) + offset)

    def limit_val(self, x, x_min, x_max):
        if x < x_min:
            x = x_min
        if x > x_max:
            x = x_max
        return x


mmc = MotorModuleController('/dev/ttyUSB1')

def print_dec2bin(dec):
    print("{0:b}".format(dec))

connceted to motor module controller


In [81]:
id = 1
p_des = 10
v_des = 20
kp = 10
kd = 0
t_ff = 0

data_pack = mmc.pack_cmd(id, p_des, v_des, kp, kd, t_ff)

# print_dec2bin(p_des)


In [82]:
#decode p_des
print_dec2bin(data_pack[1])
print_dec2bin(data_pack[2])

P = data_pack[1]<<8 | data_pack[2]

print_dec2bin(P)
mmc.uint2float(P, mmc.P_MIN, mmc.P_MAX, 16) 

11100110
1100101
1110011001100101


TypeError: int() can't convert non-string with explicit base

In [76]:
fltP = mmc.float2uint(p_des, mmc.P_MIN, mmc.P_MAX, 16)
print(fltP)
uintP = mmc.uint2float(fltP,  mmc.P_MIN, mmc.P_MAX, 16)
print(p_des, ': ',uintP)

58981
10 :  9


In [66]:
#decode v_des
print_dec2bin(data_pack[3])
print_dec2bin(data_pack[4])

V = data_pack[3]<<8 | data_pack[4]>>4

print_dec2bin(V)
mmc.uint2float(V, mmc.V_MIN, mmc.V_MAX, 16) 

10100111
1010000
1010011100000101


19.815899900816362

61865

11.099984740978105

In [69]:
fltP = mmc.float2uint(uintP, mmc.P_MIN, mmc.P_MAX, 16)
fltP

61865

In [120]:
mmc.float2uint(1.23, 0, 13, 16)

6200

In [127]:
id = 1
p_des = 10
v_des = 20
kp = 10
kd = 0
i_ff = 0

p_des = mmc.float2uint(p_des, mmc.P_MIN, mmc.P_MAX, 16)
v_des = mmc.float2uint(v_des, mmc.V_MIN, mmc.V_MAX, 12)
kp = mmc.float2uint(kp, mmc.KP_MIN, mmc.KP_MAX, 12)
kd = mmc.float2uint(kd, mmc.KD_MIN, mmc.KD_MAX, 12)
i_ff = mmc.float2uint(i_ff, -mmc.I_MAX, mmc.I_MAX, 12)

d1 = p_des >> 8
d2 = p_des&0xFF
d3 = v_des>>4
d4 = ((v_des&0xF)<<4) | (kp>>8)
d5 = kp&0xFF
d6 = kd>>4
d7 = ((kd&0xF)<<4) | i_ff>>8
d8 = i_ff&0xFF

data = bytes(bytearray([id, d1, d2, d3, d4, d5, d6, d7, d8]))
data



b'\x01\x7f\xff\x7f\xf0Q\x00\x07\xff'

In [128]:
i_ff

2047

In [59]:
pack("f", 5)

b'\x00\x00\xa0@'

In [98]:
a = 0b1111
print("{0:b}".format(a))

b = 0b01010
print("{0:b}".format(b | a))

1111
1111


In [103]:
a = 0b01010101
b = 0b11011011

"{0:b}".format(a<<8 | b)

'101010111011011'